In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

from keras.layers import Conv3D, MaxPool3D, Flatten, Dense
from keras.layers import Dropout, Input, BatchNormalization
from sklearn.metrics import confusion_matrix, accuracy_score
from plotly.offline import iplot, init_notebook_mode
from keras.losses import categorical_crossentropy
from keras.optimizers import Adadelta
import plotly.graph_objs as go
from matplotlib.pyplot import cm
from keras.models import Model
import numpy as np
import keras
import h5py

init_notebook_mode(connected=True)
%matplotlib inline
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/3d-mnist/voxelgrid.py
/kaggle/input/3d-mnist/test_point_clouds.h5
/kaggle/input/3d-mnist/plot3D.py
/kaggle/input/3d-mnist/full_dataset_vectors.h5
/kaggle/input/3d-mnist/train_point_clouds.h5


In [4]:
 with h5py.File("/kaggle/input/3d-mnist/full_dataset_vectors.h5", "r") as hf:
     X_train = hf["X_train"][:]
     y_train = hf["y_train"][:]    
     X_test = hf["X_test"][:]  
     y_test = hf["y_test"][:] 

In [5]:
print(X_train.shape,X_test.shape,y_train.shape , y_test.shape)

(10000, 4096) (2000, 4096) (10000,) (2000,)


In [6]:
x__train = np.ndarray((X_train.shape[0],4096,3))
x__test = np.ndarray((X_test.shape[0],4096,3))
def add_rgb_dimention(array):
    scaler_map = cm.ScalarMappable(cmap="Oranges")
    array = scaler_map.to_rgba(array)[:, : -1]
    return array
for i in range(X_train.shape[0]):
    x__train[i] = add_rgb_dimention(X_train[i])
for i in range(X_test.shape[0]):
    x__test[i] = add_rgb_dimention(X_test[i])

    
x_train = x__train.reshape(X_train.shape[0],16,16,16,3)
x_test = x__test.reshape(X_test.shape[0],16,16,16,3)

ytrain = keras.utils.to_categorical(y_train, 10)
ytest = keras.utils.to_categorical(y_test, 10)
print(x_train.shape,x_test.shape,ytrain.shape , ytest.shape)

(10000, 16, 16, 16, 3) (2000, 16, 16, 16, 3) (10000, 10) (2000, 10)


In [18]:
## input layer
input_layer = Input((16, 16, 16, 3))

## convolutional layers
conv_layer1 = Conv3D(filters=16, kernel_size=(3, 3, 3), activation='relu')(input_layer)
conv_layer2 = Conv3D(filters=64, kernel_size=(3, 3, 3), activation='relu')(conv_layer1)

## add max pooling to obtain the most imformatic features
pooling_layer1 = MaxPool3D(pool_size=(2, 2, 2))(conv_layer2)

conv_layer3 = Conv3D(filters=256, kernel_size=(3, 3, 3), activation='relu')(pooling_layer1)
conv_layer4 = Conv3D(filters=512, kernel_size=(3, 3, 3), activation='relu')(conv_layer3)
pooling_layer2 = MaxPool3D(pool_size=(2, 2, 2))(conv_layer4)

## perform batch normalization on the convolution outputs before feeding it to MLP architecture
pooling_layer2 = BatchNormalization()(pooling_layer2)
flatten_layer = Flatten()(pooling_layer2)

## create an MLP architecture with dense layers : 4096 -> 512 -> 10
## add dropouts to avoid overfitting / perform regularization
dense_layer1 = Dense(units=1024, activation='relu')(flatten_layer)
dense_layer1 = Dropout(0.4)(dense_layer1)
dense_layer2 = Dense(units=256, activation='relu')(dense_layer1)
dense_layer2 = Dropout(0.4)(dense_layer2)
output_layer = Dense(units=10, activation='softmax')(dense_layer2)

## define the model with input layer and output layer
model = Model(inputs=input_layer, outputs=output_layer)


In [19]:
model.compile(loss=categorical_crossentropy, optimizer=Adadelta(lr=0.1), metrics=['acc'])
model.fit(x=x_train, y=ytrain, batch_size=100, epochs=100, validation_split=0.2)


Epoch 1/100
80/80 [==============================] - 3s 36ms/step - loss: 1.8023 - acc: 0.3791 - val_loss: 2.3849 - val_acc: 0.1190
Epoch 2/100
80/80 [==============================] - 3s 33ms/step - loss: 1.2843 - acc: 0.5575 - val_loss: 2.2810 - val_acc: 0.1670
Epoch 3/100
80/80 [==============================] - 3s 34ms/step - loss: 1.0872 - acc: 0.6286 - val_loss: 2.1502 - val_acc: 0.2800
Epoch 4/100
80/80 [==============================] - 3s 32ms/step - loss: 0.9682 - acc: 0.6674 - val_loss: 1.7325 - val_acc: 0.4535
Epoch 5/100
80/80 [==============================] - 3s 32ms/step - loss: 0.8640 - acc: 0.7064 - val_loss: 2.9286 - val_acc: 0.1520
Epoch 6/100
80/80 [==============================] - 3s 32ms/step - loss: 0.7650 - acc: 0.7433 - val_loss: 3.1242 - val_acc: 0.2120
Epoch 7/100
80/80 [==============================] - 3s 33ms/step - loss: 0.6807 - acc: 0.7749 - val_loss: 3.1588 - val_acc: 0.2305
Epoch 8/100
80/80 [==============================] - 3s 33ms/step - loss: 0.

80/80 [==============================] - 3s 33ms/step - loss: 7.4419e-04 - acc: 0.9999 - val_loss: 6.7117 - val_acc: 0.4305
Epoch 63/100
80/80 [==============================] - 3s 35ms/step - loss: 0.0021 - acc: 0.9998 - val_loss: 3.3494 - val_acc: 0.5695
Epoch 64/100
80/80 [==============================] - 3s 33ms/step - loss: 7.1919e-04 - acc: 1.0000 - val_loss: 2.2368 - val_acc: 0.6570
Epoch 65/100
80/80 [==============================] - 3s 34ms/step - loss: 4.0896e-04 - acc: 1.0000 - val_loss: 2.6086 - val_acc: 0.6495
Epoch 66/100
80/80 [==============================] - 3s 32ms/step - loss: 4.1385e-04 - acc: 1.0000 - val_loss: 5.6322 - val_acc: 0.4495
Epoch 67/100
80/80 [==============================] - 3s 32ms/step - loss: 5.2312e-04 - acc: 1.0000 - val_loss: 2.1728 - val_acc: 0.6605
Epoch 68/100
80/80 [==============================] - 3s 32ms/step - loss: 5.3058e-04 - acc: 1.0000 - val_loss: 3.9803 - val_acc: 0.5840
Epoch 69/100
80/80 [==============================] - 3s 3

In [21]:
preds = model.evaluate(x = x_test, y = ytest)
print()
print ("Loss = " + str(preds[0]))
print ("Test Accuracy = " + str(preds[1]))

63/63 [==============================] - 0s 6ms/step - loss: 2.8581 - acc: 0.6400

Loss = 2.8580892086029053
Test Accuracy = 0.6399999856948853


In [22]:
pred =model.predict(x_test)
pred = np.argmax(pred, axis=1)
print(pred,y_test)
count = 0
for i in pred:
    if pred[i] != y_test[i]:
        count += 1
print('total:',len(y_test),'Wrong-pred:',count)

[7 7 2 ... 3 9 7] [7 7 2 ... 8 9 9]
total: 2000 Wrong-pred: 476
